# Semantic Kernel을 활용한 자연어 처리 및 데이터베이스 쿼리 시스템

## 1. 필요한 라이브러리 임포트

이 섹션에서는 프로젝트에 필요한 모든 라이브러리를 임포트합니다. Semantic Kernel, Azure OpenAI, 그리고 데이터베이스 연결을 위한 라이브러리들이 포함됩니다.

In [24]:
import os
import json
import pyodbc
from semantic_kernel import Kernel
from semantic_kernel.functions import kernel_function
from semantic_kernel.functions.kernel_arguments import KernelArguments
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

## 2. 데이터베이스 연결 설정

Azure SQL 데이터베이스 연결을 위한 설정을 정의합니다.

In [25]:
SQL_CONFIG = {
    "server": os.getenv("SQL_SERVER"),
    "database": os.getenv("SQL_DATABASE"),
    "username": os.getenv("SQL_USERNAME"),
    "password": os.getenv("SQL_PASSWORD"),
    "driver": os.getenv("SQL_DRIVER", "{ODBC Driver 18 for SQL Server}")
}

## 3. Semantic Kernel 초기화 및 Azure OpenAI 설정

Semantic Kernel을 초기화하고 Azure OpenAI 서비스를 추가합니다. 이는 자연어 처리 작업의 핵심이 됩니다.

In [26]:
# Semantic Kernel 초기화
kernel = Kernel()

# Azure OpenAI 서비스 추가
kernel.add_service(
    AzureChatCompletion(
        service_id="azure_open_ai",
        deployment_name=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
        endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        api_key=os.getenv("AZURE_OPENAI_API_KEY")
    )
)

## 4. NLP → SQL 변환 플러그인

이 플러그인은 자연어 입력을 SQL 쿼리로 변환합니다. Semantic Kernel의 `kernel_function` 데코레이터를 사용하여 함수를 정의합니다.

In [27]:
# Semantic Kernel의 kernel_function 데코레이터 사용
class ConvertNlpToSqlPlugin:
    @kernel_function(
        name="nlp_to_sql",
        description="자연어 설명을 바탕으로 SQL 쿼리 작성"
    )
    async def nlp_to_sql(self, user_input) -> str:
        prompt = """
        [역할]  
        당신은 SQL 전문가입니다. 주어진 요청 사항을 기반으로 정확한 SQL 쿼리를 생성하세요.  

        ---  
        요청사항: {{$request}}
        ---  

        [테이블 스키마]  
        다음 테이블을 기반으로 SQL 쿼리를 작성해야 합니다.  

        CREATE TABLE Customers (  
            CustomerID INT PRIMARY KEY,  
            Name NVARCHAR(50) NOT NULL,  
            Email NVARCHAR(50) UNIQUE,  
            JoinDate DATETIME DEFAULT GETDATE()  
        );  

        CREATE TABLE Orders (  
            OrderID INT PRIMARY KEY,  
            CustomerID INT FOREIGN KEY REFERENCES Customers(CustomerID),  
            Product NVARCHAR(50)  
                CONSTRAINT CK_Orders_Product  
                CHECK (Product IN ('노트북', '스마트폰', '태블릿')),  
            Quantity INT CHECK (Quantity > 0),  
            OrderDate DATETIME  
        );  

        - Customers 테이블은 고객 정보를 포함하며, CustomerID는 기본 키입니다.
        - Orders 테이블은 주문 정보를 저장하며, CustomerID는 Customers 테이블의 외래 키입니다.
        - Product는 '노트북', '스마트폰', '태블릿' 중 하나여야 합니다.
        - Quantity는 0보다 커야 합니다.
        - JoinDate와 OrderDate는 날짜 필드입니다.

        [규칙]  
        - 출력은 항상 유효한 SQL 쿼리여야 합니다.
        - 테이블 스키마를 준수하며, 잘못된 컬럼명을 사용하지 않습니다.
        - 불필요한 복잡성을 피하고 최적화된 SQL을 생성합니다.
        - 필요한 경우 JOIN, WHERE, GROUP BY, ORDER BY를 적절히 사용합니다.
        - 출력 형식은 SQL 코드 블록(sql ... )을 제외해야 합니다.
        - 날짜 기반 쿼리의 경우 'YYYY-MM-DD' 형식으로 비교합니다.  

        SQL 쿼리:  
        """
        # Semantic Kernel의 KernelArguments 사용
        arguments = KernelArguments(request=user_input)
        # Semantic Kernel의 invoke_prompt 메서드 사용
        raw_sql = await kernel.invoke_prompt(prompt=prompt, arguments=arguments)
        generated_sql = str(raw_sql).strip()
        print(f"[생성된 Query]\n{generated_sql}")
        return generated_sql

## 5. SQL 실행 플러그인

이 플러그인은 생성된 SQL 쿼리를 실행하고 결과를 반환합니다. 역시 `kernel_function` 데코레이터를 사용합니다.

In [28]:
# Semantic Kernel의 kernel_function 데코레이터 사용
class QueryDbPlugin:
    @kernel_function(
        name="execute_query",
        description="데이터베이스에서 SQL 쿼리 실행"
    )
    async def execute_query(self, query: str) -> list:  
        cleaned_query = query.strip()
        conn_str = f"DRIVER={SQL_CONFIG['driver']};SERVER={SQL_CONFIG['server']};DATABASE={SQL_CONFIG['database']};UID={SQL_CONFIG['username']};PWD={SQL_CONFIG['password']};Encrypt=yes;TrustServerCertificate=no;"        
        try:
            with pyodbc.connect(conn_str) as conn:
                cursor = conn.cursor()
                cursor.execute(query)
                return [dict(zip([col[0] for col in cursor.description], row)) 
                        for row in cursor.fetchall()]
        except pyodbc.Error as e:
            return print("DATABASE_ERROR", f"데이터베이스 오류: {str(e)}")

## 6. 응답 포맷팅 플러그인

이 플러그인은 데이터베이스 쿼리 결과를 사용자 친화적인 형식으로 변환합니다.


In [29]:
# Semantic Kernel의 kernel_function 데코레이터 사용
class FormatResponsePlugin:
    @kernel_function(
        name="format_data",
        description="쿼리 결과를 사용자 친화적 형식으로 변환"
    )
    async def format_data(self, db_result: str, user_input: str) -> str:
        prompt = """
        [사용자 요청]
        {{$request}}

        [쿼리 결과] 
        {{$result}}

        [변환 규칙]
        1. 기술적 용어 제거
        2. 날짜는 'YYYY년 MM월 DD일' 형식
        3. 숫자에 천 단위 구분기호 사용
        4. 주요 통계 수치 강조

        [예시]
        결과: [{'Name':'홍길동', 'OrderDate': '2024-02-10 11:00:00'}]
        응답: 홍길동 고객님은 2024년 02월 10일에 주문하셨습니다.
        """
        # Semantic Kernel의 KernelArguments 사용
        arguments = KernelArguments(request=user_input, result=db_result)
        # Semantic Kernel의 invoke_prompt 메서드 사용
        return await kernel.invoke_prompt(prompt=prompt, arguments=arguments)

## 7. 플러그인 등록

생성한 플러그인들을 Semantic Kernel에 등록합니다.


In [30]:
# Semantic Kernel의 add_plugin 메서드 사용
kernel.add_plugin(ConvertNlpToSqlPlugin(), "SqlGenerator")
kernel.add_plugin(QueryDbPlugin(), "DbExecutor")
kernel.add_plugin(FormatResponsePlugin(), "ResponseFormatter")

KernelPlugin(name='ResponseFormatter', description=None, functions={'format_data': KernelFunctionFromMethod(metadata=KernelFunctionMetadata(name='format_data', plugin_name='ResponseFormatter', description='쿼리 결과를 사용자 친화적 형식으로 변환', parameters=[KernelParameterMetadata(name='db_result', description=None, default_value=None, type_='str', is_required=True, type_object=<class 'str'>, schema_data={'type': 'string'}, include_in_function_choices=True), KernelParameterMetadata(name='user_input', description=None, default_value=None, type_='str', is_required=True, type_object=<class 'str'>, schema_data={'type': 'string'}, include_in_function_choices=True)], is_prompt=False, is_asynchronous=True, return_parameter=KernelParameterMetadata(name='return', description='', default_value=None, type_='str', is_required=True, type_object=<class 'str'>, schema_data={'type': 'string'}, include_in_function_choices=True), additional_properties={}), invocation_duration_histogram=<opentelemetry.metrics._internal

## 8. 워크플로우 실행 함수

전체 프로세스를 실행하는 함수를 정의합니다. 이 함수는 사용자 입력을 받아 SQL 생성, 쿼리 실행, 결과 포맷팅의 단계를 거칩니다.

In [31]:
# Semantic Kernel의 invoke 메서드와 get_function 메서드 사용
async def process_request(user_input):
    arguments = KernelArguments(user_input=user_input)
    
    # 1. SQL 생성 단계
    sql_result = await kernel.invoke(kernel.get_function("SqlGenerator", "nlp_to_sql"), arguments)
    sql_query = str(sql_result).strip()
    arguments = KernelArguments(query=sql_query)

    # 2. 쿼리 실행 단계
    db_result = await kernel.invoke(kernel.get_function("DbExecutor", "execute_query"), arguments)
    print(f"[Query 수행 결과]\n{db_result}")
    db_result_str = str(db_result).strip()
    arguments = KernelArguments(db_result=db_result_str, user_input=user_input)

    # 3. 결과 포맷팅 단계
    formatted_response = await kernel.invoke(kernel.get_function("ResponseFormatter", "format_data"), arguments)
    
    return formatted_response


## 9. 실행 예시
 
마지막으로, 전체 시스템을 테스트하는 실행 예시를 제공합니다.

In [32]:
user_input = "제품별 판매량을 알려줘"
print(f"[사용자 입력]\n{user_input}")
response = await process_request(user_input)
print(f"[최종 응답]\n{response}")

[사용자 입력]
제품별 판매량을 알려줘
[생성된 Query]
SELECT  
    Product,  
    SUM(Quantity) AS TotalSales  
FROM  
    Orders  
GROUP BY  
    Product;
[Query 수행 결과]
{'Product': '노트북', 'TotalSales': 1},{'Product': '스마트폰', 'TotalSales': 2},{'Product': '태블릿', 'TotalSales': 1}
[최종 응답]
판매량 정보를 다음과 같이 정리했습니다:

- **노트북:** 총 판매량 **1대**
- **스마트폰:** 총 판매량 **2대**
- **태블릿:** 총 판매량 **1대**

필요한 추가 정보가 있다면 알려주세요! 😊
